In [5]:
import os
import numpy as np
from scipy.fftpack import fft
from scipy.io import wavfile
from scipy import signal
from glob import glob
import re
import pandas as pd
import gc
from scipy.io import wavfile


from sklearn.model_selection import train_test_split

L = 16000
legal_labels = 'yes no up down left right on off stop go silence unknown'.split()

#src folders
root_path = r'..'
out_path = r'.'
model_path = r'.'
train_audio_path = '../../train/audio/'
test_audio_path = '../../test/audio/'

In [6]:
def custom_fft(y, fs):
    T = 1.0 / fs
    N = y.shape[0]
    yf = fft(y)
    xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
    # FFT is simmetrical, so we take just the first half
    # FFT is also complex, to we take just the real part (abs)
    vals = 2.0/N * np.abs(yf[0:N//2])
    return xf, vals

def log_specgram(audio, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)

In [3]:
def pad_audio(samples):
    if len(samples) >= L: return samples
    else: return np.pad(samples, pad_width=(L - len(samples), 0), mode='constant', constant_values=(0, 0))

def chop_audio(samples, L=16000, num=1000):
    for i in range(num):
        beg = np.random.randint(0, len(samples) - L)
        yield samples[beg: beg + L]

def label_transform(labels):
    nlabels = []
    for label in labels:
        if label == '_background_noise_':
            nlabels.append('silence')
        elif label not in legal_labels:
            nlabels.append('unknown')
        else:
            nlabels.append(label)
    return pd.get_dummies(pd.Series(nlabels))

In [7]:
path=train_audio_path
subdir_list = [dir for dir in os.listdir(path)]
labels_list= subdir_list
target_word_list=['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
silence_list=['_background_noise_']
unknown_label_list=[label for label in labels_list if label not in target_word_list and label not in silence_list]

In [8]:
subfolderlist = []
for x in os.listdir(train_audio_path):
    if os.path.isdir(train_audio_path + '/' + x):
        subfolderlist.append(x)

In [9]:
# obtain a file map with path and label for training iterators
file_target = []
for x in subfolderlist:
    temp = [[train_audio_path+x+'/'+y,x] for y in os.listdir(train_audio_path + x) if '.wav' in y]
    file_target += temp

train_file_map = pd.DataFrame(file_target, columns=['path', 'label'])

In [10]:
train_file_map.head()

,path,label
0,../../train/audio/bed/c245d3d7_nohash_0.wav,bed
1,../../train/audio/bed/a1a59b72_nohash_0.wav,bed
2,../../train/audio/bed/89f3ab7d_nohash_1.wav,bed
3,../../train/audio/bed/35c8fa78_nohash_1.wav,bed
4,../../train/audio/bed/1706c35f_nohash_0.wav,bed


In [11]:
train_file_map["nlabel"]=''    

In [12]:
for index,row in train_file_map.iterrows():
    if row["label"] in target_word_list:
        row["nlabel"]=row["label"]
    elif row["label"]=='_background_noise_':
        row["nlabel"]='silence'
    else:
        row["nlabel"]='unknown'

In [13]:
train_file_map.head()

,path,label,nlabel
0,../../train/audio/bed/c245d3d7_nohash_0.wav,bed,unknown
1,../../train/audio/bed/a1a59b72_nohash_0.wav,bed,unknown
2,../../train/audio/bed/89f3ab7d_nohash_1.wav,bed,unknown
3,../../train/audio/bed/35c8fa78_nohash_1.wav,bed,unknown
4,../../train/audio/bed/1706c35f_nohash_0.wav,bed,unknown


In [14]:
targets_to_keep = ['yes', 'no', 'up', 'down', 'left',
                   'right', 'on', 'off', 'stop', 'go', 'silence']
train_file_map['target'] = train_file_map['nlabel']
train_file_map['target'] = train_file_map['target'].apply(
    lambda x: x if x in targets_to_keep else 'unknown')
label_to_ix = {
    'unknown': 11,
    'silence': 0,
    'down': 1,
    'go': 2,
    'left': 3,
    'no': 4,
    'off': 5,
    'on': 6,
    'right': 7,
    'stop': 8,
    'up': 9,
    'yes': 10}
ix_to_label ={
    11: 'unknown',
    0: 'silence',
    1: 'down',
    2: 'go',
    3: 'left',
    4: 'no',
    5: 'off',
    6: 'on',
    7: 'right',
    8: 'stop',
    9: 'up',
    10: 'yes'
}
train_file_map['label'] = train_file_map['target'].apply(lambda x: label_to_ix[x])

In [15]:
train_file_map=train_file_map.drop(columns="nlabel")

In [16]:
train_file_map.tail()

,path,label,target
64722,../../train/audio/zero/2ad772d6_nohash_1.wav,11,unknown
64723,../../train/audio/zero/9ff2d2f4_nohash_0.wav,11,unknown
64724,../../train/audio/zero/30f31e42_nohash_0.wav,11,unknown
64725,../../train/audio/zero/37dca74f_nohash_1.wav,11,unknown
64726,../../train/audio/zero/0d393936_nohash_0.wav,11,unknown


In [17]:
train_file_map.to_csv("train_file_map_csv.csv", index=True)

In [18]:
train_file_map["target"].unique()

array(['unknown', 'right', 'on', 'stop', 'no', 'off', 'silence', 'up',
       'yes', 'go', 'left', 'down'], dtype=object)

In [19]:
new_sample_rate=16000
y_train = []
x_train = []
for i in range(train_file_map.shape[0]):
    sample_rate, samples = wavfile.read(train_file_map["path"][i])
    samples = pad_audio(samples)
    if len(samples) > 16000:
        n_samples = chop_audio(samples)
    else: n_samples = [samples]
    for samples in n_samples:
        resampled = signal.resample(samples, int(new_sample_rate / sample_rate * samples.shape[0]))
        _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
        y_train.append(train_file_map["label"][i])
        x_train.append(specgram)
x_train = np.array(x_train)
x_train = x_train.reshape(tuple(list(x_train.shape)+[1]))
y_train= np.array(y_train)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [20]:
print('min: ',np.min(x_train), 
      '\nmax: ', np.max(x_train), 
      '\nmean: ', np.mean(x_train),
      '\nmedian: ', np.median(x_train),
      '\nvariance: ', np.var(x_train))

min:  -23.02585 
max:  16.476927 
mean:  -3.101561 
median:  -3.2602808 
variance:  36.329456


In [22]:
np.save('../../train/x_train.npy', x_train)
np.save('../../train/y_train.npy', y_train.astype(np.int))